#### Import Libraries

In [1]:
import torch
import json
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.preprocessor import PreProcessor
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http, eval_data_from_json
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
from haystack.document_store import InMemoryDocumentStore
from haystack.retriever.sparse import TfidfRetriever
from haystack.utils import print_answers
from haystack.reader.farm import FARMReader
from haystack.pipeline import ExtractiveQAPipeline

06/22/2021 22:42:04 - INFO - faiss.loader -   Loading faiss with AVX2 support.
06/22/2021 22:42:04 - INFO - faiss.loader -   Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
06/22/2021 22:42:04 - INFO - faiss.loader -   Loading faiss.
06/22/2021 22:42:06 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
06/22/2021 22:42:08 - INFO - faiss.loader -   Loading faiss with AVX2 support.
06/22/2021 22:42:08 - INFO - faiss.loader -   Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
06/22/2021 22:42:08 - INFO - faiss.loader -   Loading faiss.


#### Check Cuda

In [2]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce GTX 1050


#### Connective to ElasticSearchDocumentStore

In [3]:
# Connect to Elasticsearch
document_store = ElasticsearchDocumentStore(
        host="localhost",
        port=9200,
        text_field = 'text',
        name_field = 'name',
        username="",
        password="",
        index="document")

06/22/2021 22:42:09 - INFO - elasticsearch -   HEAD http://localhost:9200/ [status:200 request:0.028s]
06/22/2021 22:42:09 - INFO - elasticsearch -   HEAD http://localhost:9200/document [status:200 request:0.011s]
06/22/2021 22:42:09 - INFO - elasticsearch -   GET http://localhost:9200/document [status:200 request:0.004s]
06/22/2021 22:42:09 - INFO - elasticsearch -   PUT http://localhost:9200/document/_mapping [status:200 request:0.020s]
06/22/2021 22:42:09 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.003s]


In [4]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [5]:
# Loading trained model

reader = FARMReader('my_model')

06/22/2021 22:42:09 - INFO - farm.utils -   Using device: CUDA 
06/22/2021 22:42:09 - INFO - farm.utils -   Number of GPUs: 1
06/22/2021 22:42:09 - INFO - farm.utils -   Distributed Training: False
06/22/2021 22:42:09 - INFO - farm.utils -   Automatic Mixed Precision: None
06/22/2021 22:42:11 - WARNING - farm.modeling.prediction_head -   Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"training": true, "num_labels": 2, "ph_output_type": "per_token_squad", "model_type": "span_classification", "label_tensor_name": "question_answering_label_ids", "label_list": ["start_token", "end_token"], "metric": "squad", "name": "QuestionAnsweringHead"}
06/22/2021 22:42:15 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
06/22/2021 22:42:15 - INFO - farm.utils -   Using device: CUDA 
06/22/2021 22:42:15 - INFO - farm.utils -   Number of GPUs: 1
06/22/2021 22:42:15 - INFO - farm.utils -   

In [6]:
# Creating a pipeline

pipe = ExtractiveQAPipeline(reader, retriever)

#### Prediction Funcion

In [8]:
top_k_retriever=5
top_k_reader=3
def get_answer(question, top_answer= False):
    prediction = pipe.run(query=question, top_k_retriever=top_k_retriever, top_k_reader=top_k_reader)
    if len(prediction['answers']) == 0:
        return "No answers found"
    print("Top Answer = ", prediction['answers'][0]['answer'])
    print("-"* 100)
    answers = prediction['answers']
    if not top_answer:
        for answer in answers:
            ans = answer["answer"]
            context = answer["context"]
            document_name = answer["meta"]["name"]
            score = answer["score"]
            print("Answer: \n", ans)
            print("\nContext: \n", context)
            print("\nDocument_name: \n", document_name)
            print("\nScore: \n", score)
            print('-' * 100)
    return prediction['answers'][0]['answer']

In [9]:
get_answer("Do senior students get any priority?")

06/22/2021 22:42:15 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.109s]
Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 35.74 Batches/s]

Top Answer =  Students at a more senior level with similar merits shall have a higher chance of allocation.
----------------------------------------------------------------------------------------------------
Answer: 
 Students at a more senior level with similar merits shall have a higher chance of allocation.

Context: 
 s for last competitive spot)
Students at a more senior level with similar merits shall have a higher chance of allocation. Students with prior oversea

Document_name: 
 allocation_criteria.txt

Score: 
 11.602807998657227
----------------------------------------------------------------------------------------------------
Answer: 
 Please do not make any payment fee for any administrative matter including visa, air tickets and accommodation until you are advised to do so.

Context: 
 Please do not make any payment fee for any administrative matter including visa, air tickets and accommodation until you are advised to do so.

Document_name: 
 dates_and_deadlines.txt

S

'Students at a more senior level with similar merits shall have a higher chance of allocation.'

In [12]:
from tkinter import *

def printData(firstName, lastName):
    print(firstName)
    print(lastName)
    root.destroy()

def print_answer():
    question = entry1.get()
    answer = get_answer(question, False)
    answer_label.config(text="Answer: {}".format(answer))


root = Tk()

#question label
label1 = Label(root,text = 'Enter your question')
label1.pack()
label1.config(justify = CENTER)

entry1 = Entry(root, width = 100)
entry1.pack()

button1 = Button(root, text = 'submit', command = print_answer)
button1.pack() 
button1.config(command = print_answer)

answer_label = Label(root,text = '')
answer_label.pack()
answer_label.config(justify = CENTER)
    
root.mainloop()

06/22/2021 23:49:14 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.005s]
